<a href="https://colab.research.google.com/github/Yanina-Kutovaya/ccfd/blob/main/notebooks/01_Feature_engineering_Featuretools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Cards Fraud Detection
## Feature engineering with Featuretools
Data: https://www.kaggle.com/competitions/geekbrains-competitive-data-analysis/data 

## 1. Install featuretools

In [1]:
#!pip install featuretools

## 2. Get data from kaggle

In [2]:
# upload kaggle.json to /content/
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c geekbrains-competitive-data-analysis

 94% 112M/119M [00:01<00:00, 132MB/s] 
100% 119M/119M [00:01<00:00, 98.5MB/s]


In [3]:
!mkdir data
!unzip geekbrains-competitive-data-analysis.zip -d data

Archive:  geekbrains-competitive-data-analysis.zip
  inflating: data/applications_history.csv  
  inflating: data/bki.csv            
  inflating: data/client_profile.csv  
  inflating: data/payments.csv       
  inflating: data/sample_submit.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


## 3. Importing libraries

In [4]:
import pandas as pd
import featuretools as ft
from featuretools import selection 
from woodwork.logical_types import (
    Categorical, AgeFractional, Ordinal, BooleanNullable
)
import pickle

## 4. Uploading data

In [5]:
PATH = 'data/'
applications_history = pd.read_csv(PATH + 'applications_history.csv')
bki = pd.read_csv(PATH + 'bki.csv')
client_profile = pd.read_csv(PATH + 'client_profile.csv')
payments = pd.read_csv(PATH + 'payments.csv')
sample_submit = pd.read_csv(PATH + 'sample_submit.csv')
test = pd.read_csv(PATH + 'test.csv')
train = pd.read_csv(PATH + 'train.csv')

In [6]:
data = [
    applications_history,
    bki,
    client_profile,
    payments, 
    sample_submit,
    test,
    train
]
for item in data:
  item.columns =  map(str.lower, item.columns)
  
client_profile['age'] = client_profile['age'] / 365

## 5. Dataset overview

In [7]:
print(f'applications_history.shape = {applications_history.shape}\n')
applications_history.head(2)

applications_history.shape = (1670214, 26)



,prev_application_number,application_number,name_contract_type,amount_annuity,amt_application,amount_credit,amount_payment,amount_goods_payment,name_contract_status,days_decision,...,name_product_type,sellerplace_area,cnt_payment,name_yield_group,days_first_drawing,days_first_due,days_last_due_1st_version,days_last_due,days_termination,nflag_insured_on_approval
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,...,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,...,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0


In [8]:
print(f'bki.shape = {bki.shape}\n')
bki.head(2)

bki.shape = (945234, 17)



,application_number,bureau_id,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.81,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.00,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN


In [9]:
print(f'client_profile.shape = {client_profile.shape}\n')
client_profile.head(2)

client_profile.shape = (250000, 24)



,application_number,gender,childrens,total_salary,amount_credit,amount_annuity,education_level,family_status,region_population,age,...,family_size,external_scoring_rating_1,external_scoring_rating_2,external_scoring_rating_3,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,23.452055,...,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,63.526027,...,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0


In [10]:
print(f'payments.shape = {payments.shape}\n')
payments.head(2)

payments.shape = (1023932, 8)



,prev_application_number,application_number,num_instalment_version,num_instalment_number,days_instalment,days_entry_payment,amt_instalment,amt_payment
0,49011181,123664960,1.0,5,1002.0,1015.0,12156.615,12156.615
1,48683432,123497205,1.0,13,442.0,432.0,18392.535,10047.645


In [11]:
print(f'sample_submit.shape = {sample_submit.shape}\n')
sample_submit.head(2)

sample_submit.shape = (165141, 2)



,application_number,target
0,123724268,0
1,123456549,0


In [12]:
print(f'test.shape = {test.shape}\n')
test.head(2)

test.shape = (165141, 2)



,application_number,name_contract_type
0,123724268,Cash
1,123456549,Cash


In [13]:
print(f'train.shape = {train.shape}\n')
train.head(2)

train.shape = (110093, 3)



,application_number,target,name_contract_type
0,123687442,0,Cash
1,123597908,1,Cash


## 6. Representing Data with EntitySets

### 6.1 Creating an EntitySet

In [14]:
es = ft.EntitySet(id='app')

### 6.2 Adding dataframes

In [15]:
app = pd.concat([train, test], ignore_index=True, sort=False)
income_order = ['XNA', 'low_action', 'low_normal', 'middle', 'high']
es = es.add_dataframe(
    dataframe_name="applications",
    dataframe=app,
    index="application_number",        
)
es = es.add_dataframe(
    dataframe_name="applications_history", 
    dataframe=applications_history, 
    index="prev_application_number",
    logical_types={
        "name_yield_group": Ordinal(order=income_order),
        "nflag_insured_on_approval": BooleanNullable,
    }            
)
es = es.add_dataframe(
    dataframe_name="bki", 
    dataframe=bki,
    index="index",
)
education_order = [
    'Lower secondary', 'Secondary / secondary special',  'Incomplete higher', 
    'Higher education', 'Academic degree'
]
es = es.add_dataframe(
    dataframe_name="client_profile", 
    dataframe=client_profile, 
    index="index",
    logical_types={
        "education_level": Ordinal(order=education_order),
        "age": AgeFractional,
    }
)
es = es.add_dataframe(
    dataframe_name="payments", 
    dataframe=payments,
    index="index"
)
es

/usr/local/lib/python3.8/dist-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/featuretools/entityset/entityset.py:1906: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(


Entityset: app
  DataFrames:
    applications [Rows: 275234, Columns: 3]
    applications_history [Rows: 1670214, Columns: 26]
    bki [Rows: 945234, Columns: 18]
    client_profile [Rows: 250000, Columns: 25]
    payments [Rows: 1023932, Columns: 9]
  Relationships:
    No relationships

### 6.3 Adding a Relationship

In [16]:
es = es.add_relationship(
    "applications", "application_number", "applications_history", "application_number"
)
es = es.add_relationship(
    "applications", "application_number", "bki", "application_number"
)
es = es.add_relationship(
    "applications", "application_number", "client_profile", "application_number"
)
es = es.add_relationship(
    "applications", "application_number", "payments", "application_number"
)
es = es.add_relationship(
    "applications_history", "prev_application_number", "payments", "prev_application_number"
)
es

Entityset: app
  DataFrames:
    applications [Rows: 275234, Columns: 3]
    applications_history [Rows: 1670214, Columns: 26]
    bki [Rows: 945234, Columns: 18]
    client_profile [Rows: 250000, Columns: 25]
    payments [Rows: 1023932, Columns: 9]
  Relationships:
    applications_history.application_number -> applications.application_number
    bki.application_number -> applications.application_number
    client_profile.application_number -> applications.application_number
    payments.application_number -> applications.application_number
    payments.prev_application_number -> applications_history.prev_application_number

## 7. Feature Engineering

In [17]:
ft.list_primitives()

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,any,aggregation,True,False,Determines if any value is 'True' in a list.,"<ColumnSchema (Logical Type = Boolean)>, <Colu...",<ColumnSchema (Logical Type = Boolean)>
1,mean,aggregation,True,True,Computes the average for a list of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
2,last,aggregation,False,False,Determines the last value in a list.,<ColumnSchema>,None
3,entropy,aggregation,False,False,Calculates the entropy for a categorical column,<ColumnSchema (Semantic Tags = ['category'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
4,trend,aggregation,False,False,Calculates the trend of a column over time.,"<ColumnSchema (Semantic Tags = ['numeric'])>, ...",<ColumnSchema (Semantic Tags = ['numeric'])>
...,...,...,...,...,...,...,...
149,is_quarter_start,transform,True,True,Determines the is_quarter_start attribute of a...,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = BooleanNullable)>
150,lag,transform,False,False,Shifts an array of values by a specified numbe...,"<ColumnSchema (Semantic Tags = ['category'])>,...",None
151,num_unique_separators,transform,False,False,Calculates the number of unique separators.,<ColumnSchema (Logical Type = NaturalLanguage)>,<ColumnSchema (Logical Type = IntegerNullable)...
152,subtract_numeric_scalar,transform,True,True,Subtracts a scalar from each element in the list.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>


In [18]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es, 
    target_dataframe_name="applications",
    max_features=1000,
    chunk_size=4000,
    verbose=True,
    max_depth=3,
    n_jobs=-1
)
feature_matrix

Built 816 features
EntitySet scattered to 2 workers in 29 seconds
Elapsed: 2:11:56 | Progress:  87%|████████▋ 

distributed.nanny - WARNING - Restarting worker


Elapsed: 3:33:06 | Progress:  95%|█████████▌

distributed.nanny - WARNING - Worker process still alive after 3.9999973297119142 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3.9999977111816407 seconds, killing


Elapsed: 3:33:44 | Progress: 100%|██████████


,target,name_contract_type,COUNT(applications_history),MAX(applications_history.amount_annuity),MAX(applications_history.amount_credit),MAX(applications_history.amount_goods_payment),MAX(applications_history.amount_payment),MAX(applications_history.amt_application),MAX(applications_history.cnt_payment),MAX(applications_history.days_decision),...,STD(applications_history.MIN(payments.applications.target)),STD(applications_history.NUM_UNIQUE(payments.applications.name_contract_type)),STD(applications_history.SKEW(payments.applications.target)),STD(applications_history.SUM(payments.applications.target)),SUM(applications_history.MAX(payments.applications.target)),SUM(applications_history.MEAN(payments.applications.target)),SUM(applications_history.MIN(payments.applications.target)),SUM(applications_history.NUM_UNIQUE(payments.applications.name_contract_type)),SUM(applications_history.SKEW(payments.applications.target)),SUM(applications_history.STD(payments.applications.target))
application_number,,,,,,,,,,,,,,,,,,,,,
123687442,0,Cash,3,9262.215,70758.0,72634.14,7267.14,72634.14,12.0,2702.0,...,NaN,NaN,NaN,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
123597908,1,Cash,4,38667.600,722569.5,495000.00,0.00,495000.00,48.0,903.0,...,NaN,NaN,NaN,0.500000,1.0,1.0,1.0,1.0,0.0,0.0
123526683,0,Cash,6,53956.800,1527579.0,1395000.00,18814.50,1395000.00,48.0,2829.0,...,0.0,0.0,NaN,0.000000,0.0,0.0,0.0,3.0,0.0,0.0
123710391,1,Cash,2,6025.275,100858.5,100858.50,4500.00,100858.50,18.0,1904.0,...,NaN,NaN,NaN,0.707107,1.0,1.0,1.0,1.0,0.0,0.0
123590329,1,Cash,9,26347.455,793251.0,697500.00,5850.00,697500.00,60.0,1522.0,...,NaN,NaN,NaN,1.333333,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123487967,<NA>,Cash,6,25409.745,545040.0,450000.00,NaN,450000.00,48.0,278.0,...,NaN,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
123536402,<NA>,Cash,2,24301.935,218700.0,243000.00,24300.00,243000.00,10.0,404.0,...,NaN,NaN,NaN,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
123718238,<NA>,Cash,10,29164.140,491580.0,450000.00,0.00,450000.00,24.0,2874.0,...,NaN,0.0,NaN,0.000000,0.0,0.0,0.0,4.0,0.0,0.0


## 8. Feature selection

In [19]:
feature_matrix, features = ft.selection.remove_single_value_features(
    feature_matrix, features=feature_defs
)
feature_matrix = ft.selection.remove_highly_null_features(feature_matrix)

## 9. Save artifacts

In [20]:
applications_history.to_parquet('applications_history.parquet.gzip', compression='gzip')
bki.to_parquet('bki.parquet.gzip', compression='gzip')
client_profile.to_parquet('client_profile.parquet.gzip', compression='gzip')
payments.to_parquet('payments.parquet.gzip', compression='gzip')
sample_submit.to_parquet('sample_submit.parquet.gzip', compression='gzip')
test.to_parquet('test.parquet.gzip', compression='gzip')
train.to_parquet('train.parquet.gzip', compression='gzip')

feature_matrix.to_parquet('feature_matrix.parquet.gzip', compression='gzip')

In [21]:
feature_matrix.shape[1]

737